In [1]:
from convert_ast import from_dict,to_dict,file_to_dict
from localization import Localization
from pycparser import parse_file, c_parser, c_generator, c_ast
from my_tools import go_to_func,get_type,get_fuzzer_result

In [4]:
file_dict = file_to_dict("test.c")

In [ ]:
text = r"""
int test(char* data,int size){
  int a;
  int*b;
  int c=1;
  struct node *x;
  x->v=10+*(data)-'0';
  a=c+func(x->v,c);
  a=a+1;
  *b=a;
  a=0;
  c=a;
  return 0;
}

int func(int arg1,char* arg2){
  return arg1;
}
"""

In [ ]:
ast = parser.parse(text)
dic=to_dict(ast)
dic

In [ ]:
cur_dict,loc=Localization(file_dict,12,5).loc(1)


cur_dict.insert(loc,patch)

file_ast = from_dict(file_dict)
generator = c_generator.CGenerator()

print(generator.visit(file_ast))

In [ ]:
file_dict

In [ ]:
parser = c_parser.CParser() # パーサ
ast = parser.parse(text)

In [ ]:
patch={'_nodetype': 'FuncCall',
      'coord': ':4:3',
      'name': {'_nodetype': 'ID', 'name': 'printf', 'coord': ':4:3'},
      'args': {'_nodetype': 'ExprList',
       'coord': ':4:10',
       'exprs': [{'_nodetype': 'Constant',
         'type': 'string',
         'value': '"num: %d\\n"',
         'coord': ':4:10'},
        {'_nodetype': 'ID', 'name': 'data', 'coord': ':4:22'}]}}

In [ ]:
file_dict

In [ ]:
'''
check input dependency of a function
input: Dependency(file_dict,funcname,arg_index)
output: jsonfile{funcname, variable, type,line, col}

only consider of int

todo: 
1. branch condition has dependent variables
2. array
3. recursive

function not define
'''
import json
class Dependency:
    def __init__(self,dic):
        self.dic=dic
        self.funcname=None
        self.dep={}#information of depent variables
        self.res={}#result
        self.func={}#location of functions
        self.var={}#type of variables
        self.varname=None#for return_spec
        self.spec={
            "If":self.if_spec,
            "While":self.while_spec,
            "For":self.for_spec,
            "FuncCall":self.funccall_spec,
            "Assignment":self.assignment_spec,
            "Decl":self.decl_spec,
            "Return":self.return_spec
        }
        self.ext=self.dic["ext"]
        for i in range(len(self.ext)):
            if self.ext[i]["_nodetype"]=="FuncDef":
                self.func[self.ext[i]["decl"]["name"]]=i
                self.dep[self.ext[i]["decl"]["name"]]=set()
                self.var[self.ext[i]["decl"]["name"]]={}

    def collect_dep(self,funcname,index):
        self.call_func(funcname,index)
        return json.dumps(self.res,indent=4) 
        
    def call_func(self,funcname,index):
        #print("funcname: "+funcname)
        #variable name
        varname=self.ext[self.func[funcname]]["decl"]["type"]["args"]["params"][index]["name"]
        self.varname=varname
        #initialize in res
        try:
            if self.ext[self.func[funcname]]["decl"]["type"]["args"]["params"][index]["type"]["type"]["names"][0]=='int':
                vartype="int"
            else:
                vartype="others"
        except:
            vartype="others"
        #add var to self.var
        self.var[funcname][varname]=vartype
        #add var to self.res   
        if funcname in self.res:
            if varname in self.res[funcname]["args"].keys():
                return self.res[funcname]["retvaldep"][varname]
            self.res[funcname]["args"][varname]=vartype
            self.res[funcname]["retvaldep"][varname]=False
        else:
            self.res[funcname]={"args":{varname:vartype},
                                "coord":self.ext[self.func[funcname]]["coord"],
                                "retvaldep":{varname:False},
                                "dep":[]
                               }
        #add to self.dep
        self.dep[funcname].add(varname)

            
        self.funcname=funcname
        for s in self.ext[self.func[funcname]]["body"]["block_items"]:
            if s["_nodetype"] not in self.spec:
                print({s["_nodetype"]} + " not in spec")
                continue
            else:
                self.spec[s["_nodetype"]](s)
                self.funcname=funcname
                self.varname=varname
                
        return self.res[funcname]["retvaldep"][varname]
        

    def if_spec(self,cur_dic):
        pass
        #print("If")
    def while_spec(self,cur_dic):
        pass
        #print("While")
    def for_spec(self,cur_dic):
        pass
        #print("For")
    def funccall_spec(self,cur_dic):
        pass
        #print("FuncCall")
    def assignment_spec(self,cur_dic):
        #FuncCall,UnaryOp,BinaryOp,"ID","Constant"
        #left value
        var=""
        l_dict=cur_dic["lvalue"]
        while True:
            if l_dict["_nodetype"]=="ID":
                var+=l_dict["name"]
                coord=l_dict["coord"]
                break
            elif l_dict["_nodetype"]=="UnaryOp":
                var+="*"
                l_dict=l_dict["expr"]
            elif l_dict["_nodetype"]=="StructRef":
                var+=l_dict["name"]["name"]+l_dict["type"]+l_dict["field"]["name"]
                coord=l_dict["field"]["coord"]
                break
        
        if self.check_assignment(cur_dic["rvalue"]):
            #add to self.dep
            self.dep[self.funcname].add(var)
            #check type
            if "->" in var:return
            if "int" in self.var[self.funcname][var]:
                star_num = len(self.var[self.funcname][var].split())-1
                self.res[self.funcname]["dep"].append(("*"*star_num+var,coord))
        else:
            if var in self.dep[self.funcname]:
                self.dep[self.funcname].remove(var)

        #print("Assignment")
    def decl_spec(self,cur_dic):
        vartype=""
        s=cur_dic["type"]
        while True:
            if s["_nodetype"]=="PtrDecl":
                vartype+=" *"
                s=s["type"]
            elif s["_nodetype"]=="TypeDecl":
                var=s["declname"]
                coord=s["coord"]
                s=s["type"]
            elif s["_nodetype"]=="IdentifierType":
                vartype=s["names"][0]+vartype
                break
            elif s["_nodetype"]=="Struct":
                vartype="struct "+s["name"]+vartype
                break
        #add var to self.var
        self.var[self.funcname][var]=vartype
        for i in range(len(vartype.split())-1,-1,-1):
            if vartype.split()[i]=="*":
                self.var[self.funcname]["*"+var]=" ".join(vartype.split()[:i])

        if cur_dic["init"]:
            if self.check_assignment(cur_dic["init"]):
                #add to self.dep
                self.dep[self.funcname].add(var)
                #check type
                if "int" in self.var[self.funcname][var]:
                    star_num = len(self.var[self.funcname][var].split())-1
                    self.res[self.funcname]["dep"].append(("*"*star_num+var,coord))
            else:
                if var in self.dep[self.funcname]:
                    self.dep[self.funcname].remove(var)
              
        #print("Decl")
        
    def return_spec(self,cur_dic):
        if self.check_assignment(cur_dic["expr"]):
            self.res[self.funcname]["retvaldep"][self.varname]=True
        #print("return")
        
        
    def check_assignment(self,cur_dic):
        #check func args
        if cur_dic["_nodetype"]=="FuncCall":
            if cur_dic["name"]["name"] == self.funcname:
                return False
            #print("in funccall")
            try:
                for i in range(len(cur_dic["args"]["exprs"])):
                    #print("name "+cur_dic["name"]["name"])
                    if self.check_assignment(cur_dic["args"]["exprs"][i]):
                        tmp=self.funcname
                        res=self.call_func(cur_dic["name"]["name"],i)
                        self.funcname=tmp
                        return res
            except:
                return False
        var=""
        tmp=cur_dic
        while tmp["_nodetype"] in ["UnaryOp","StructRef","ID"]:
            if tmp["_nodetype"]=="ID":
                var+=tmp["name"]
                return var in self.dep[self.funcname] or tmp["name"] in self.dep[self.funcname]
            elif tmp["_nodetype"]=="UnaryOp":
                var+="*"
                tmp=tmp["expr"]
            elif tmp["_nodetype"]=="StructRef":
                var+=tmp["name"]["name"]+tmp["type"]+tmp["field"]["name"]
                coord=tmp["field"]["coord"]
                return var in self.dep[self.funcname]
        #check other types         
        for i in cur_dic.keys():
            if type(cur_dic[i]) is dict:
                if self.check_assignment(cur_dic[i]): 
                    return True 
            if type(cur_dic[i]) is list:
                for e in cur_dic[i]:
                    if type(e) is dict and self.check_assignment(e): 
                        return True                     
        return False

In [ ]:
print(Dependency(file_dict).collect_dep("test",0))

In [ ]:
f=open("out.json","w")
json.dump(Dependency(dic).collect_dep("test",0),f)

In [ ]:
x=[1,2,3]

In [ ]:
x.remove(1)

In [ ]:
x

In [ ]:
get_type(go_to_func(file_dict,43,"p11_constant_reverse")["decl"]["type"]["args"]["params"][0]["type"],"")

In [5]:
go_to_func(file_dict,43,"p11_constant_reverse")

{'_nodetype': 'FuncDef',
 'coord': 'test.c:35:1',
 'decl': {'_nodetype': 'Decl',
  'name': 'p11_constant_reverse',
  'quals': [],
  'storage': [],
  'funcspec': [],
  'coord': 'test.c:35:1',
  'type': {'_nodetype': 'FuncDecl',
   'coord': 'test.c:35:1',
   'args': {'_nodetype': 'ParamList',
    'coord': 'test.c:35:27',
    'params': [{'_nodetype': 'Decl',
      'name': 'nick',
      'quals': [],
      'storage': [],
      'funcspec': [],
      'coord': 'test.c:35:27',
      'type': {'_nodetype': 'TypeDecl',
       'declname': 'nick',
       'quals': [],
       'coord': 'test.c:35:27',
       'type': {'_nodetype': 'IdentifierType',
        'names': ['int'],
        'coord': 'test.c:35:23'}},
      'bitsize': None,
      'init': None}]},
   'type': {'_nodetype': 'PtrDecl',
    'quals': [],
    'coord': 'test.c:34:10',
    'type': {'_nodetype': 'TypeDecl',
     'declname': 'p11_constant_reverse',
     'quals': [],
     'coord': 'test.c:35:1',
     'type': {'_nodetype': 'IdentifierType',
 

In [2]:
get_fuzzer_result("fuzzing_result")

[{'coord': 'infer.c:43:15',
  'funcname': 'p11_constant_reverse',
  'next': {'coord': 'infer.c:17:12',
   'funcname': 'p11_dict_new',
   'next': {'coord': 'a.out+0x52204d)', 'funcname': 'malloc', 'next': {}}}},
 {'coord': 'infer.c:43:15',
  'funcname': 'p11_constant_reverse',
  'next': {'coord': 'infer.c:19:40',
   'funcname': 'p11_dict_new',
   'next': {'coord': 'a.out+0x52204d)', 'funcname': 'malloc', 'next': {}}}}]

In [17]:
def get_dependency(fuzzer,filename):
    err_path=get_fuzzer_result(fuzzer)
    file_dict = file_to_dict(filename)
    dependency=[]
    for err in err_path:
        var={}
        while err["next"]:
            line=int(err["coord"].split(":")[-2])
            funcname=err["funcname"]
            func_dict=go_to_func(file_dict,line,funcname)
            var[funcname]=Dependency(func_dict).out()
            err=err["next"]
        dependency.append(var)
    return dependency

In [20]:
go_to_func(file_dict,17,"p11_dict_new")

{'_nodetype': 'FuncDef',
 'coord': 'test.c:10:1',
 'decl': {'_nodetype': 'Decl',
  'name': 'p11_dict_new',
  'quals': [],
  'storage': [],
  'funcspec': [],
  'coord': 'test.c:10:1',
  'type': {'_nodetype': 'FuncDecl',
   'coord': 'test.c:10:1',
   'type': {'_nodetype': 'PtrDecl',
    'quals': [],
    'coord': 'test.c:9:10',
    'type': {'_nodetype': 'TypeDecl',
     'declname': 'p11_dict_new',
     'quals': [],
     'coord': 'test.c:10:1',
     'type': {'_nodetype': 'IdentifierType',
      'names': ['p11_dict'],
      'coord': 'test.c:9:1'}}},
   'args': None},
  'bitsize': None,
  'init': None},
 'body': {'_nodetype': 'Compound',
  'coord': 'test.c:10:1',
  'block_items': [{'_nodetype': 'Decl',
    'name': 'dict',
    'quals': [],
    'storage': [],
    'funcspec': [],
    'coord': 'test.c:11:14',
    'type': {'_nodetype': 'PtrDecl',
     'quals': [],
     'coord': 'test.c:11:14',
     'type': {'_nodetype': 'TypeDecl',
      'declname': 'dict',
      'quals': [],
      'coord': 'test

In [22]:
get_dependency("fuzzing_result","test.c")

[{'p11_constant_reverse': [{'name': 'nick',
    'type': 'int',
    'coord': 'test.c:35:27'}],
  'p11_dict_new': []},
 {'p11_constant_reverse': [{'name': 'nick',
    'type': 'int',
    'coord': 'test.c:35:27'}],
  'p11_dict_new': []}]

In [21]:
class Dependency:
    def __init__(self,dic):
        self.dic=dic
        self.dep=[]#information of depent variables
        self.cur_dep=[]
        self.spec={
            "If":self.if_spec,
            "While":self.while_spec,
            "For":self.for_spec,
            "FuncCall":self.funccall_spec,
            "Assignment":self.assignment_spec,
            "Decl":self.decl_spec,
            "Return":self.return_spec
        }
        try:
            for arg in dic["decl"]["type"]["args"]["params"]:
                param={
                    "name":arg["name"],
                    "type":get_type(arg["type"],""),
                    "coord":arg["coord"]
                }
                self.cur_dep.append(arg["name"])
                self.dep.append(param)
        except:
            pass
    def out(self):
        return self.dep
            

    def collect_dep(self,funcname,index):
        self.call_func(funcname,index)
        return json.dumps(self.res,indent=4) 
        
    def call_func(self,funcname,index):
        #print("funcname: "+funcname)
        #variable name
        varname=self.ext[self.func[funcname]]["decl"]["type"]["args"]["params"][index]["name"]
        self.varname=varname
        #initialize in res
        try:
            if self.ext[self.func[funcname]]["decl"]["type"]["args"]["params"][index]["type"]["type"]["names"][0]=='int':
                vartype="int"
            else:
                vartype="others"
        except:
            vartype="others"
        #add var to self.var
        self.var[funcname][varname]=vartype
        #add var to self.res   
        if funcname in self.res:
            if varname in self.res[funcname]["args"].keys():
                return self.res[funcname]["retvaldep"][varname]
            self.res[funcname]["args"][varname]=vartype
            self.res[funcname]["retvaldep"][varname]=False
        else:
            self.res[funcname]={"args":{varname:vartype},
                                "coord":self.ext[self.func[funcname]]["coord"],
                                "retvaldep":{varname:False},
                                "dep":[]
                               }
        #add to self.dep
        self.dep[funcname].add(varname)

            
        self.funcname=funcname
        for s in self.ext[self.func[funcname]]["body"]["block_items"]:
            if s["_nodetype"] not in self.spec:
                print({s["_nodetype"]} + " not in spec")
                continue
            else:
                self.spec[s["_nodetype"]](s)
                self.funcname=funcname
                self.varname=varname
                
        return self.res[funcname]["retvaldep"][varname]
        

    def if_spec(self,cur_dic):
        pass
        #print("If")
    def while_spec(self,cur_dic):
        pass
        #print("While")
    def for_spec(self,cur_dic):
        pass
        #print("For")
    def funccall_spec(self,cur_dic):
        pass
        #print("FuncCall")
    def assignment_spec(self,cur_dic):
        #FuncCall,UnaryOp,BinaryOp,"ID","Constant"
        #left value
        var=""
        l_dict=cur_dic["lvalue"]
        while True:
            if l_dict["_nodetype"]=="ID":
                var+=l_dict["name"]
                coord=l_dict["coord"]
                break
            elif l_dict["_nodetype"]=="UnaryOp":
                var+="*"
                l_dict=l_dict["expr"]
            elif l_dict["_nodetype"]=="StructRef":
                var+=l_dict["name"]["name"]+l_dict["type"]+l_dict["field"]["name"]
                coord=l_dict["field"]["coord"]
                break
        
        if self.check_assignment(cur_dic["rvalue"]):
            #add to self.dep
            self.dep[self.funcname].add(var)
            #check type
            if "->" in var:return
            if "int" in self.var[self.funcname][var]:
                star_num = len(self.var[self.funcname][var].split())-1
                self.res[self.funcname]["dep"].append(("*"*star_num+var,coord))
        else:
            if var in self.dep[self.funcname]:
                self.dep[self.funcname].remove(var)

        #print("Assignment")
    def decl_spec(self,cur_dic):
        vartype=""
        s=cur_dic["type"]
        while True:
            if s["_nodetype"]=="PtrDecl":
                vartype+=" *"
                s=s["type"]
            elif s["_nodetype"]=="TypeDecl":
                var=s["declname"]
                coord=s["coord"]
                s=s["type"]
            elif s["_nodetype"]=="IdentifierType":
                vartype=s["names"][0]+vartype
                break
            elif s["_nodetype"]=="Struct":
                vartype="struct "+s["name"]+vartype
                break
        #add var to self.var
        self.var[self.funcname][var]=vartype
        for i in range(len(vartype.split())-1,-1,-1):
            if vartype.split()[i]=="*":
                self.var[self.funcname]["*"+var]=" ".join(vartype.split()[:i])

        if cur_dic["init"]:
            if self.check_assignment(cur_dic["init"]):
                #add to self.dep
                self.dep[self.funcname].add(var)
                #check type
                if "int" in self.var[self.funcname][var]:
                    star_num = len(self.var[self.funcname][var].split())-1
                    self.res[self.funcname]["dep"].append(("*"*star_num+var,coord))
            else:
                if var in self.dep[self.funcname]:
                    self.dep[self.funcname].remove(var)
              
        #print("Decl")
        
    def return_spec(self,cur_dic):
        if self.check_assignment(cur_dic["expr"]):
            self.res[self.funcname]["retvaldep"][self.varname]=True
        #print("return")
        
        
    def check_assignment(self,cur_dic):
        #check func args
        if cur_dic["_nodetype"]=="FuncCall":
            if cur_dic["name"]["name"] == self.funcname:
                return False
            #print("in funccall")
            try:
                for i in range(len(cur_dic["args"]["exprs"])):
                    #print("name "+cur_dic["name"]["name"])
                    if self.check_assignment(cur_dic["args"]["exprs"][i]):
                        tmp=self.funcname
                        res=self.call_func(cur_dic["name"]["name"],i)
                        self.funcname=tmp
                        return res
            except:
                return False
        var=""
        tmp=cur_dic
        while tmp["_nodetype"] in ["UnaryOp","StructRef","ID"]:
            if tmp["_nodetype"]=="ID":
                var+=tmp["name"]
                return var in self.dep[self.funcname] or tmp["name"] in self.dep[self.funcname]
            elif tmp["_nodetype"]=="UnaryOp":
                var+="*"
                tmp=tmp["expr"]
            elif tmp["_nodetype"]=="StructRef":
                var+=tmp["name"]["name"]+tmp["type"]+tmp["field"]["name"]
                coord=tmp["field"]["coord"]
                return var in self.dep[self.funcname]
        #check other types         
        for i in cur_dic.keys():
            if type(cur_dic[i]) is dict:
                if self.check_assignment(cur_dic[i]): 
                    return True 
            if type(cur_dic[i]) is list:
                for e in cur_dic[i]:
                    if type(e) is dict and self.check_assignment(e): 
                        return True                     
        return False

In [10]:
Dependency(go_to_func(file_dict,43,"p11_constant_reverse")).out()

[{'name': 'nick', 'type': 'int', 'coord': 'test.c:35:27'}]